# Basic DfT REST API

- Inspo: https://www.tomforth.co.uk/toomanybuses/
- Docs: https://data.bus-data.dft.gov.uk/api/buslocation-openapi/#/SIRI-VM%20Data%20feed/get_datafeed

In [1]:
import json
import os

import httpx
import xmltodict
from dotenv import load_dotenv
from lxml import etree

In [2]:
load_dotenv()

True

## Overall settings

In [3]:
base_url = "https://data.bus-data.dft.gov.uk/api/v1/"
api_key = os.environ["API_KEY"]
adminArea = 340

## Schedules
Not actually useful to do this, easier to just bulk download the GTFS files

In [4]:
client = httpx.Client(base_url=base_url, params={"api_key": api_key})

In [5]:
r = client.get(
    "dataset/", params={"adminArea": adminArea, "limit": 500, "search": "Oxford"}
)
r.status_code

200

In [6]:
results = r.json()["results"]
len(results)

221

In [7]:
r = results[7]
r["id"], r["operatorName"], r["name"], r["lines"], r["url"]

(265,
 'Arriva UK Bus',
 'Arriva UK Bus_High Wycombe_Amersham_20200601',
 ['1', '1A', '30', '31', '32', '33', '37', '37A', '41', '48', '800', '850'],
 'https://data.bus-data.dft.gov.uk/timetable/dataset/265/download/')

In [8]:
# results[175]

In [9]:
# sorted((r["name"], i) for i, r in enumerate(results))

## Location
Rather use the bods-client Python library

In [10]:
client = httpx.Client(base_url=base_url, params={"api_key": api_key})

In [13]:
def get_activity_list(line_ref, op_ref) -> list[dict]:
    r = client.get("datafeed/", params={"lineRef": line_ref, "operatorRef": op_ref})
    assert r.status_code == 200
    d = xmltodict.parse(r.text)
    va = d["Siri"]["ServiceDelivery"]["VehicleMonitoringDelivery"]["VehicleActivity"]
    # print(json.dumps(va, indent=2))
    return va

In [14]:
lines = (
    get_activity_list("5", "OXBC")
    + get_activity_list("1", "SCOX")
    + get_activity_list("10", "SCOX")
)
len(lines)

23

In [15]:
lines[:2]

[{'RecordedAtTime': '2022-10-15T08:37:25+00:00',
  'ItemIdentifier': 'e206b5ee-0dc8-4b92-9de9-143de202f740',
  'ValidUntilTime': '2022-10-15T08:42:50.588232',
  'MonitoredVehicleJourney': {'LineRef': '5',
   'DirectionRef': 'inbound',
   'FramedVehicleJourneyRef': {'DataFrameRef': '2022-10-15',
    'DatedVehicleJourneyRef': '5043'},
   'PublishedLineName': '5',
   'OperatorRef': 'OXBC',
   'OriginRef': '340001230ERB',
   'OriginName': 'Blackbird_Leys__Sandy_Lane_West',
   'DestinationRef': '340000006R2',
   'DestinationName': 'Oxford_City_Centre__Railway_Station',
   'OriginAimedDepartureTime': '2022-10-15T09:20:00+00:00',
   'DestinationAimedArrivalTime': '2022-10-15T10:09:00+00:00',
   'VehicleLocation': {'Longitude': '-1.204361', 'Latitude': '51.718875'},
   'Bearing': '290.0',
   'Occupancy': 'seatsAvailable',
   'BlockRef': '13512',
   'VehicleRef': '657'},
  'Extensions': {'VehicleJourney': {'Operational': {'TicketMachine': {'TicketMachineServiceCode': '5',
      'JourneyCode': '